Matrix element (weakfield) - 2nd version
========
Testenv to check the valitity of the matrix element calculation on a grid.

In [ ]:
from __future__ import print_function
import sftrident as sf
import sftrident.coordinate as corr
from sftrident.matrixElement2 import matrixElementWF
import numpy as np
import time
times = {}

In [ ]:
def buildArr_lcc(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return [ss,p1m,p1x,p1y,p2m,p2x,p2y]


mass = 1.0
Narr1 = (2,2,2,2,2,2,2)
frame = 'lab'
tempDPHI = 50.0

arrLCC = buildArr_lcc(*Narr1)

start = time.time()
kinObj = corr.momenta(*arrLCC,mass = mass,frame = frame,coordSys='lcc')
end =  time.time() - start
times['initMom']=end

config = {'a0':1e-4,'mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}

start = time.time()
matObj = matrixElementWF(kinObj,**config)
end =  time.time() - start
times['initMat'] = end

print(matObj.bw(0,1,1,0)[0])



In [ ]:
spins = [(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 0, 1, 1), (1, 1, 0, 0), (1, 1, 0, 1), (1, 1, 1, 0), (1, 1, 1, 1)]

wrappBW = lambda spn : np.asarray(matObj.bw(*spn))

start = time.time()
resBWgrid = np.array(map(wrappBW,spins))
end =  time.time() - start
times['mapMat'] =end

print(resBWgrid.shape)

In [ ]:
def sorted2List(X,Y):
    """
    sorts the list X with the values of Y
    """
    return [x for _, x in sorted(zip(Y,X), key=lambda pair: pair[0])]


modeDict={
'bw':[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15],
'c':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
'bwx':[0, 11, 8, 3, 14, 5, 6, 13, 2, 9, 10, 1, 12, 7, 4, 15],
'cx':[0, 14, 2, 12, 11, 5, 9, 7, 8, 6, 10, 4, 3, 13, 1, 15]
}
spinMode = np.array([
    (0,0,0,0),
    (0,0,1,1),
    (0,0,0,1),
    (0,0,1,0),
    (1,1,0,0),
    (1,1,1,1),
    (1,1,0,1),
    (1,1,1,0),
    (0,1,0,0),
    (0,1,1,1),
    (0,1,0,1),
    (0,1,1,0),
    (1,0,0,0),
    (1,0,1,1),
    (1,0,0,1),
    (1,0,1,0),
])
    
matObjSQ = sf.matrixElement(config)
def getMatBW(n):
    kinPara = [kinObj.grid.ssGrid[n],kinObj.grid.lccGrid2[1][n], kinObj.grid.lccGrid2[2][n],kinObj.grid.lccGrid2[0][n],kinObj.grid.lccGrid1[1][n], kinObj.grid.lccGrid1[2][n],kinObj.grid.lccGrid1[0][n]]
    matObjSQ.setKin(kinPara)
    resOld = matObjSQ.evalMdirPartBW()
    tempSpins = spinMode[modeDict['bw']]
    #print(sorted2List([list(t) for t in tempSpins],[list(t) for t in tempSpins]))
    return sorted2List(np.asarray(resOld),[list(t) for t in tempSpins])
    #return kinPara

err = 0
maxAERR_R = 0.0
maxAERR_I = 0.0

relERR_R = 0.0
maxRERR_R = 0.0
relERR_I = 0.0
maxRERR_I = 0.0
for ind in np.arange(kinObj.grid.size):
    bwResSQ = getMatBW(ind)
    MsqSQ = 0.0
    MsqNP = 0.0
    for i,spin in enumerate(spins):
        #print('i',i)
        #print()
        #print(el)
        #print("\t",bwResSQ[i])
        #print("\t",resBWgrid[i][ind])
        test = np.isclose(bwResSQ[i],resBWgrid[i][ind],atol=1e-15,rtol=1e-8)
        absERR_R = bwResSQ[i].real-resBWgrid[i][ind].real
        maxAERR_R = max(maxAERR_R,absERR_R)
        norm_R = bwResSQ[i].real+resBWgrid[i][ind].real
        if norm_R != 0.0:
            relERR_R = absERR_R/norm_R
        maxRERR_R = max(maxRERR_R,relERR_R)
        
        absERR_I = bwResSQ[i].imag-resBWgrid[i][ind].imag
        maxAERR_I = max(maxAERR_I,absERR_I)
        norm_I = bwResSQ[i].imag+resBWgrid[i][ind].imag
        if norm_I != 0.0:
            relERR_I = absERR_I/norm_I
        maxRERR_I = max(maxRERR_I,relERR_I)
        if not test:
            #print("ind",ind)
            #print('i',i)
            #print(spin)
            #print(spins)
            #print("\t",bwResSQ[i])
            #print("\t",resBWgrid[i][ind])
            err+=1
        MsqSQ += np.real(bwResSQ[i]*np.conjugate(bwResSQ[i]))
        MsqNP += np.real(resBWgrid[i][ind]*np.conjugate(resBWgrid[i][ind]))
    print(ind)
    print("\t",MsqSQ)
    print("\t",MsqNP)
        
print("Done with %d errors."%err)
print("max. abs. error: %1.2e + i %1.2e"%(maxAERR_R,maxAERR_I))
print("max. rel. error: %1.2e + i %1.2e"%(maxRERR_R,maxRERR_I))

In [ ]:
from __future__ import print_function
import numpy as np
from sftrident import current2
from sftrident import coordinate
from sftrident import alpha
from sftrident.weakField import fsquare
from sftrident.kinematics import photoNumBW
from sftrident import matrixElement

arrLCC = buildArr_lcc(*Narr1)
kinObj = coordinate.momenta(*arrLCC,mass = mass,frame = frame,coordSys='lcc')
J0c = current2.J0(kinObj.P1,kinObj.P,kinObj.K,mode='c') # compton part
J0bw = current2.J0(kinObj.P2,kinObj.P3,kinObj.K,mode='bw') # bw part
J1bw = current2.J1(kinObj.P2,kinObj.P3,kinObj.K,mode='bw') # bw part

tempDPHI = 50.0
config = {'a0':1e-4,'mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}
alphaFKTbw = alpha([kinObj.K,kinObj.P2,kinObj.P3],config)
alpha1bw = alphaFKTbw(1)


F = fsquare().getF() #change name to  F!!
Fsq = lambda s : F(s,config['dPhi'])

#k,-p,p1,p2,p3

pNum = photoNumBW(0.0,[kinObj.K,-kinObj.P,kinObj.P1,kinObj.P2,kinObj.P3])

def prodCurr(curr1,curr2):
    def temp(s11,s12,s21,s22):
        return curr1(0,s11,s12)*curr2(0,s21,s22) - curr1(1,s11,s12)*curr2(1,s21,s22)-curr1(2,s11,s12)*curr2(2,s21,s22)-curr1(3,s11,s12)*curr2(3,s21,s22)
    return temp

def M11(s1,s,s2,s3):
    termBW = lambda mu,xs2,xs3: J1bw(mu,xs2,xs3) - alpha1bw/pNum*J0bw(mu,xs2,xs3)
    tempProd = prodCurr(J0c,termBW)
    #print(tempProd(s1,s,s2,s3))
    return 2.0*np.pi*tempProd(s1,s,s2,s3)*Fsq(pNum)/((kinObj.P1 - kinObj.P)*(kinObj.P1 - kinObj.P))

print(M11(0, 0, 1, 1)[3])

In [ ]:
for el in times.iterkeys():
    print(el,times[el]/(kinObj.grid.size*1.0))

Conclusion
===

There are some issues with the spin mapping in sftrident.matrixElement. The results are (mostly) exactly the same, but with a overall minus. Therefore, the squared matrixElement is not influenced from this bug. But there need to be some research, where the bug comes from and how one may solve it. 

Overall, one may conclude, that the BW matrix element is computed with a grid evaluation and gives almost the same results as the serial evaluation (modulo sign). 